# Mask R-CNN - Train on Shapes Dataset

### Notes from implementation

This notebook shows how to train Mask R-CNN on your own dataset. To keep things simple we use a synthetic dataset of shapes (squares, triangles, and circles) which enables fast training. You'd still need a GPU, though, because the network backbone is a Resnet101, which would be too slow to train on a CPU. On a GPU, you can start to get okay-ish results in a few minutes, and good results in less than an hour.



In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys
import random
import math
import re
import  gc
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras.backend as KB
sys.path.append('../')

import mrcnn.model_mod as modellib
import mrcnn.visualize as visualize
import mrcnn.shapes    as shapes
from mrcnn.config      import Config
from mrcnn.model       import log
from mrcnn.dataset     import Dataset 

from mrcnn.utils       import stack_tensors, stack_tensors_3d
from mrcnn.datagen     import data_generator, load_image_gt
from mrcnn.callbacks   import get_layer_output_1,get_layer_output_2
from mrcnn.visualize   import plot_gaussian

# Root directory of the project
ROOT_DIR = os.getcwd()
MODEL_PATH = 'E:\Models'
# Directory to save logs and trained model
MODEL_DIR = os.path.join(MODEL_PATH, "mrcnn_development_logs")

# Path to COCO trained weights
COCO_MODEL_PATH   = os.path.join(MODEL_PATH, "mask_rcnn_coco.h5")
RESNET_MODEL_PATH = os.path.join(MODEL_PATH, "resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")

print("Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))


# Build configuration object -----------------------------------------------
config = shapes.ShapesConfig()
config.BATCH_SIZE      = 5                  # Batch size is 2 (# GPUs * images/GPU).
config.IMAGES_PER_GPU  = 5                  # Must match BATCH_SIZE
config.STEPS_PER_EPOCH = 2
config.FCN_INPUT_SHAPE = config.IMAGE_SHAPE[0:2]

# config.LAST_EPOCH_RAN  = 5784
config.display() 

# Build shape dataset        -----------------------------------------------
# Training dataset
# generate 500 shapes 
dataset_train = shapes.ShapesDataset()
dataset_train.load_shapes(500, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_train.prepare()

# Validation dataset
dataset_val = shapes.ShapesDataset()
dataset_val.load_shapes(50, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_val.prepare()

# Load and display random samples
# image_ids = np.random.choice(dataset_train.image_ids, 3)
# for image_id in [3]:
#     image = dataset_train.load_image(image_id)
#     mask, class_ids = dataset_train.load_mask(image_id)
#     visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)
print(' COCO Model Path       : ', COCO_MODEL_PATH)
print(' Checkpoint folder Path: ', MODEL_DIR)
print(' Model Parent Path     : ', MODEL_PATH)
print(' Resent Model Path     : ', RESNET_MODEL_PATH)

import pprint
pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4)


D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Tensorflow Version: 1.6.0   Keras Version : 2.1.4 
 Initialize config object - super
(56, 56)

Configuration Parameters:
-------------------------
BACKBONE_SHAPES                [[32 32]
 [16 16]
 [ 8  8]
 [ 4  4]
 [ 2  2]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     5
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
EPOCHS_TO_RUN                  0
FCN_INPUT_SHAPE                [128 128]
GPU_COUNT                      1
IMAGES_PER_GPU                 5
IMAGE_MAX_DIM                  128
IMAGE_MIN_DIM                  128
IMAGE_PADDING                  True
IMAGE_SHAPE                    [128 128   3]
LAST_EPOCH_RAN                 0
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [12

# Create Model

In [2]:
try :
    del model
    gc.collect()
except: 
    pass
KB.clear_session()
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

>>> Initialize model WITHOUT MASKING LAYERS!!!!
    set_log_dir: Checkpoint path set to : E:\Models\mrcnn_development_logs\shapes20180513T1945\mask_rcnn_shapes_{epoch:04d}.h5
    set_log_dir: self.epoch set to 0 

>>> Resnet Graph 
     Input_image shape : (?, 128, 128, 3)
     After ZeroPadding2D  : (?, 134, 134, 3) (?, 134, 134, 3)
     After Conv2D padding : (?, 64, 64, 64) (?, 64, 64, 64)
     After BatchNorm      : (?, 64, 64, 64) (?, 64, 64, 64)
     After MaxPooling2D   : (?, 32, 32, 64) (?, 32, 32, 64)

>>> Feature Pyramid Network (FPN) Graph 
     FPN P2 shape : (None, 32, 32, 256)
     FPN P3 shape : (None, 16, 16, 256)
     FPN P4 shape : (None, 8, 8, 256)
     FPN P5 shape : (None, 4, 4, 256)
     FPN P6 shape : (None, 2, 2, 256)

>>> RPN Layer 
     Input_feature_map shape : (?, ?, ?, 256)
     anchors_per_location    : 3
     depth                   : 256
     Input_feature_map shape : (?, ?, ?, 256)
     anchors_per_location    : 3
     anchor_stride           : 1

>>> R

   FCN Block 31 shape is :  (5, 32, 32, 256)
   FCN Block 32 shape is :  (5, 32, 32, 256)
   FCN Block 33 shape is :  (5, 32, 32, 256)
   FCN Block 34 (Max pooling) shape is :  (5, 16, 16, 256)
   FCN fully connected 1 (fcn_fc1) shape is :  (5, 16, 16, 1024)
   FCN fully connected 2 (fcn_fc2) shape is :  (5, 16, 16, 1024)
   FCN final conv2d (fcn_classify) shape is :  (None, 16, 16, 4)
   h_factor :  8.0 w_factor :  8.0

>>> BilinearUpSampling2D layer
     data_format :  channels_last
     size        :  (8.0, 8.0)
     target_size :  None
     input_spec  :  [InputSpec(ndim=4)]
     call resize_images_bilinear with size:  (8.0, 8.0)
     CHANNELS LAST: X:  (5, 16, 16, 4)  KB.int_shape() :  (None, 16, 16, 4)
     target_height   :  None  target_width  :  None
     new_shape (2):  (2,) (2,)
     new_shape (3):  (2,) (2,)
     X after image.resize_bilinear:  (5, ?, ?, 4)
     Dimensions of X after set_shape() :  (5, 128, 128, 4)
    BilinearUpSampling2D. compute_output_shape()
   FCN hea

In [3]:
print(' COCO Model Path       : ', COCO_MODEL_PATH)
print(' Checkpoint folder Path: ', MODEL_DIR)
print(' Model Parent Path     : ', MODEL_PATH)
print(' Resent Model Path     : ', RESNET_MODEL_PATH)
print(model.find_last())

# model.compile_only(learning_rate=config.LEARNING_RATE, layers='all')
# tst = model.keras_model.to_json()
# save_model(MODEL_DIR, 'my_saved_model')
# print(model.find_last())

 COCO Model Path       :  E:\Models\mask_rcnn_coco.h5
 Checkpoint folder Path:  E:\Models\mrcnn_development_logs
 Model Parent Path     :  E:\Models
 Resent Model Path     :  E:\Models\resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
>>> find_last checkpoint in :  E:\Models\mrcnn_development_logs
(None, None)


In [4]:
#model.keras_model.summary(line_length = 120) 
# model.compile_only(learning_rate=config.LEARNING_RATE, layers='heads')
# KB.set_learning_phase(1)
'''
methods to load weights
1 - load a specific file
2 - find a last checkpoint in a specific folder 
3 - use init_with keyword 
'''
## 1- look for a specific weights file 
## Load trained weights (fill in path to trained weights here)
# model_path  = 'E:\\Models\\mrcnn_logs\\shapes20180428T1819\\mask_rcnn_shapes_5784.h5'
# print(' model_path : ', model_path )
# assert model_path != "", "Provide path to trained weights"
# print("Loading weights from ", model_path)
# model.load_weights(model_path, by_name=True)    
# print('Load weights complete')

# ## 2- look for last checkpoint file in a specific folder (not working correctly)
# model.config.LAST_EPOCH_RAN = 5784
# model.model_dir = 'E:\\Models\\mrcnn_logs\\shapes20180428T1819'
# last_model_found = model.find_last()
# print(' last model in MODEL_DIR: ', last_model_found)
# # loc= model.load_weights(model.find_last()[1], by_name=True)
# # print('Load weights complete :', loc)


## 3- Use init_with keyword
## Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
#     loc=model.load_weights(model.get_imagenet_weights(), by_name=True)
    loc=model.load_weights(RESNET_MODEL_PATH, by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    
    # See README for instructions to download the COCO weights
    loc=model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    loc= model.load_weights(model.find_last()[1], by_name=True)
print('Load weights complete')

>>> load_weights()
    load_weights: Loading weights from: E:\Models\mask_rcnn_coco.h5
    load_weights: Log directory set to : E:\Models\mask_rcnn_coco.h5
    set_log_dir: Checkpoint path set to : E:\Models\mrcnn_development_logs\shapes20180513T1946\mask_rcnn_shapes_{epoch:04d}.h5
    set_log_dir: self.epoch set to 0 
    Load weights complete :  E:\Models\mask_rcnn_coco.h5
Load weights complete


###  Print some model information 

In [5]:
print('\n Inputs: ') 
for i, out in enumerate(model.keras_model.inputs):
    print(i , '    ', out)

print('\n Outputs: ') 
for i, out in enumerate(model.keras_model.outputs):
    print(i , '    ', out)

# print('\n Losses (model.metrics_names): ') 
# pp.pprint(model.get_deduped_metrics_names())
# model.keras_model.summary(line_length = 150) 


 Inputs: 
0      Tensor("input_image:0", shape=(?, 128, 128, 3), dtype=float32)
1      Tensor("input_image_meta:0", shape=(?, ?), dtype=float32)
2      Tensor("input_rpn_match:0", shape=(?, ?, 1), dtype=int32)
3      Tensor("input_rpn_bbox:0", shape=(?, ?, 4), dtype=float32)
4      Tensor("input_gt_class_ids:0", shape=(?, ?), dtype=int32)
5      Tensor("input_gt_boxes:0", shape=(?, ?, 4), dtype=float32)
6      Tensor("input_gt_masks:0", shape=(?, 56, 56, ?), dtype=bool)

 Outputs: 
0      Tensor("rpn_class_logits/concat:0", shape=(?, ?, 2), dtype=float32)
1      Tensor("rpn_class/concat:0", shape=(?, ?, 2), dtype=float32)
2      Tensor("rpn_bbox/concat:0", shape=(?, ?, 4), dtype=float32)
3      Tensor("rpn_proposal_rois/packed_2:0", shape=(5, ?, ?), dtype=float32)
4      Tensor("proposal_targets/output_rois:0", shape=(5, ?, ?), dtype=float32)
5      Tensor("proposal_targets/target_class_ids:0", shape=(5, ?), dtype=int32)
6      Tensor("proposal_targets/target_bbox_deltas:0", shape=(5,

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

## Training head using  Keras.model.fit_generator()

In [ ]:
print(config.BATCH_SIZE)
print(model.config.BATCH_SIZE)

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.

# model.train(dataset_train, dataset_val, 
#             learning_rate=config.LEARNING_RATE, 
# #             epochs = 69,
#             epochs_to_run =2, 
#             layers='heads')

train_layers = ['mrcnn', 'fpn','rpn']
loss_names   = [  "rpn_class_loss", "rpn_bbox_loss" , "mrcnn_class_loss", "mrcnn_bbox_loss"]


model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs = 25,
#             epochs_to_run =0, 
#             batch_size = 0
#             steps_per_epoch = 0 
            layers = train_layers,
            losses= loss_names
            )

['mrcnn', 'fpn', 'rpn']
['(mrcnn\\_.*)', '(fpn\\_.*)', '(rpn\\_.*)']
layers regex : (mrcnn\_.*)|(fpn\_.*)|(rpn\_.*)

Selecting layers to train
-------------------------
Layer    Layer Name               Layer Type
   0  input_image            (InputLayer          )   ............................no weights to train ]
   1  zero_padding2d_1       (ZeroPadding2D       )   ............................no weights to train ]
   2  conv1                  (Conv2D              )   ............................not a layer we want to train ]
   3  bn_conv1               (BatchNorm           )   ............................not a layer we want to train ]
   4  activation_1           (Activation          )   ............................no weights to train ]
   5  max_pooling2d_1        (MaxPooling2D        )   ............................no weights to train ]
   6  res2a_branch2a         (Conv2D              )   ............................not a layer we want to train ]
   7  bn2a_branch2a          (B

Starting at epoch 0 of 25 epochs. LR=0.001

Steps per epochs 2 
Batch size       5 
Checkpoint Path: E:\Models\mrcnn_development_logs\shapes20180513T1946\mask_rcnn_shapes_{epoch:04d}.h5 


D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\tensorflow\python\ops\gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
2/2 [==============================] - 10s 5s/step - loss: 4.7420 - rpn_class_loss: 0.0268 - rpn_bbox_loss: 1.2504 - mrcnn_class_loss: 2.0219 - mrcnn_bbox_loss: 1.4430 - val_loss: 3.3343 - val_rpn_class_loss: 0.0338 - val_rpn_bbox_loss: 0.9163 - val_mrcnn_class_loss: 1.1383 - val_mrcnn_bbox_loss: 1.2459

Epoch 00001: val_loss improved from inf to 3.33425665, saving model to E:\Models\mrcnn_development_logs\shapes20180513T1946\mask_rcnn_shapes_0001.h5
Epoch 2/25
2/2 [==============================] - 2s 949ms/step - loss: 3.6199 - rpn_class_loss: 0.0353 - rpn_bbox_loss: 1.1016 - mrcnn_class_loss: 1.1087 - mrcnn_bbox_loss: 1.3743 - val_loss: 2.4863 - val_rpn_class_loss: 0.0328 - val_rpn_bbox_loss: 0.8649 - val_mrcnn_class_loss: 0.5532 - val_mrcnn_bbox_loss: 1.0354

Epoch 00002: val_loss improved from 3.33425665 to 2.48634696, saving model to E:\Models\mrcnn_development_logs\shapes20180513T1946\mask_rcnn_shapes_0002.h5


### Define Data Generator

In [ ]:
train_generator = data_generator(dataset_train, model.config, shuffle=True,
                                 batch_size=model.config.BATCH_SIZE,
                                 augment = False)
val_generator = data_generator(dataset_val, model.config, shuffle=True, 
                                batch_size=model.config.BATCH_SIZE,
                                augment=False)

### Get next shapes from generator and display loaded shapes

In [ ]:
train_batch_x, train_batch_y = next(train_generator)

In [ ]:
# train_batch_x, train_batch_y = next(train_generator)
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])
    print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)



### Push Data thru model using get_layer_output()

In [ ]:

model_output = get_layer_output_2(model.keras_model, train_batch_x, 1)


### Input Values

In [ ]:
# 0      Tensor("input_image:0", shape=(?, 128, 128, 3), dtype=float32)
# 1      Tensor("input_image_meta:0", shape=(?, ?), dtype=float32)
# 2      Tensor("input_rpn_match:0", shape=(?, ?, 1), dtype=int32)
# 3      Tensor("input_rpn_bbox:0", shape=(?, ?, 4), dtype=float32)
# 4      Tensor("input_gt_class_ids:0", shape=(?, ?), dtype=int32)
# 5      Tensor("input_gt_boxes:0", shape=(?, ?, 4), dtype=float32)
# 6      Tensor("input_gt_masks:0", shape=(?, 56, 56, ?), dtype=bool)

input_image_meta =  train_batch_x[1]
input_rpn_match  =  train_batch_x[2]
input_rpn_bbox   =  train_batch_x[3]
input_gt_class_ids = train_batch_x[4]
input_gt_bboxes = train_batch_x[5]
# gt_masks   =  train_batch_x[6]
print(' input_rpn_match    ', input_rpn_match.shape)
print(' input_rpn_bbox     ', input_rpn_bbox.shape)
print(' input_gt_class_ids ', input_gt_class_ids.shape)
print(' input_gt_bboxes    ', input_gt_bboxes.shape)


### Output Values

In [ ]:
# KB.set_session(sess)
print(len(model_output))
# rpn_class_logits   = model_output[0]
# rpn_class          = model_output[1]
# rpn_bbox           = model_output[2]
# rpn_proposal_rois  = model_output[3]
output_rois        = model_output[4]
target_class_ids   = model_output[5]
target_bbox_deltas = model_output[6]
roi_gt_boxes       = model_output[7]
mrcnn_class_logits = model_output[8]
mrcnn_class        = model_output[9]
mrcnn_bbox         = model_output[10]
# rpn_class_loss   = model_output[11]
# rpn_bbox_loss    = model_output[12]
# mrcnn_class_loss = model_output[13]
# mrcnn_bbox_loss  = model_output[14]
fcn_bbox_loss      = model_output[15]
pred_hm            = model_output[16]
gt_hm              = model_output[17]
pred_hm_norm       = model_output[18]
gt_hm_norm         = model_output[19]
pred_tensor        = model_output[20]
gt_tensor          = model_output[21]
gt_deltas          = model_output[22]
fcn_heatmap        = model_output[23]
fcn_class_logits   = model_output[24]
fcn_scores         = model_output[25]
fcn_bbox_deltas    = model_output[26]
# print(type(model_output[4]))
# print(type(output_rois))

In [ ]:
print(gt_deltas.shape, fcn_bbox_deltas.shape)

In [ ]:
img = 0
cls = 3
print(gt_deltas[img,cls])
print(fcn_bbox_deltas[img,cls])

In [ ]:
model.keras_model.summary(line_length=132, positions=[0.30,0.75, .83, 1. ])

In [ ]:
print(model.keras_model.summary)

## - Training heads using train_in_batches ()

We need to use this method for the time being as the fit generator does not have provide EASY access to the output in Keras call backs. By training in batches, we pass a batch through the network, pick up the generated RoI detections and bounding boxes and generate our semantic / gaussian tensors ...


In [ ]:
model.train_in_batches(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE/6, 
            epochs_to_run = 3,
            layers='heads')

## Simulate one training iteration - 1

In [ ]:
from mrcnn.datagen import data_generator, load_image_gt
np.set_printoptions(linewidth=100)

learning_rate   = model.config.LEARNING_RATE
epochs_to_run   = 2
layers          = 'heads'
batch_size      = 0
steps_per_epoch = 0

# assert self.mode == "training", "Create model in training mode."
# Pre-defined layer regular expressions
layer_regex = {
    # all layers but the backbone
    "heads": r"(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)|(fcn\_.*)",
    # From a specific Resnet stage and up
    "3+": r"(res3.*)|(bn3.*)|(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
    "4+": r"(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
    "5+": r"(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
    # All layers
    "all": ".*",
}

if layers in layer_regex.keys():
    layers = layer_regex[layers]
if batch_size == 0 :
    batch_size = model.config.BATCH_SIZE            
if steps_per_epoch == 0:
    steps_per_epoch = model.config.STEPS_PER_EPOCH

# Data generators
train_generator = data_generator(dataset_train, model.config, shuffle=True,
                                 batch_size=batch_size)
val_generator   = data_generator(dataset_val, model.config, shuffle=True,
                                 batch_size=batch_size,
                                 augment=False)

# Train
log("Last epoch completed : {} ".format(model.epoch))
log("Starting from epoch {} for {} epochs. LR={}".format(model.epoch, epochs_to_run, learning_rate))
log("Steps per epoch:    {} ".format(steps_per_epoch))
log("Batchsize      :    {} ".format(batch_size))
log("Checkpoint Folder:  {} ".format(model.checkpoint_path))
epochs = model.epoch + epochs_to_run

from tensorflow.python.platform import gfile
if not gfile.IsDirectory(model.log_dir):
    log('Creating checkpoint folder')
    gfile.MakeDirs(model.log_dir)
else:
    log('Checkpoint folder already exists')

model.set_trainable(layers)            
model.compile(learning_rate, model.config.LEARNING_MOMENTUM)        

out_labels = model.keras_model._get_deduped_metrics_names()
callback_metrics = out_labels + ['val_' + n for n in out_labels]

progbar = keras.callbacks.ProgbarLogger(count_mode='steps')
progbar.set_model(model.keras_model)
progbar.set_params({
    'epochs': epochs,
    'steps': steps_per_epoch,
    'verbose': 1,
    'do_validation': False,
    'metrics': callback_metrics,
})

progbar.set_model(model.keras_model) 

chkpoint = keras.callbacks.ModelCheckpoint(model.checkpoint_path, 
                                           monitor='loss', verbose=1, save_best_only = True, save_weights_only=True)
chkpoint.set_model(model.keras_model)

progbar.on_train_begin()
epoch_idx = model.epoch

## Simulate one training iteration - 2

In [ ]:
if epoch_idx >= epochs:
    print('Final epoch {} has already completed - Training will not proceed'.format(epochs))

# while epoch_idx < epochs :
progbar.on_epoch_begin(epoch_idx)
steps_index = 0
# for steps_index in range(steps_per_epoch):

batch_logs = {}
print(' self.epoch {}   epochs {}  step {} '.format(model.epoch, epochs, steps_index))
batch_logs['batch'] = steps_index
batch_logs['size']  = batch_size
progbar.on_batch_begin(steps_index, batch_logs)

In [ ]:
train_batch_x, train_batch_y = next(train_generator)

## Simulate one training iteration - 3

In [ ]:
imgmeta_idx= model.keras_model.input_names.index('input_image_meta')
img_meta  =  train_batch_x[imgmeta_idx]

image_id = img_meta[0,0]
print('Image id: ',image_id)
image = dataset_train.load_image(image_id)
mask, class_ids = dataset_train.load_mask(image_id)
visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

image_id = img_meta[1,0]
print('Image id: ',image_id)
image = dataset_train.load_image(image_id)
mask, class_ids = dataset_train.load_mask(image_id)
visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

outs = model.keras_model.train_on_batch(train_batch_x, train_batch_y)

### Simulate one training iteration - 4

In [ ]:
if not isinstance(outs, list):
    outs = [outs]
for l, o in zip(out_labels, outs):
    batch_logs[l] = o

progbar.on_batch_end(steps_index, batch_logs)

        # print(outs)
progbar.on_epoch_end(epoch_idx, {})
    # if (epoch_idx % 10) == 0:
chkpoint.on_epoch_end(epoch_idx  , batch_logs)
epoch_idx += 1

# if epoch_idx != self.epoch:
# chkpoint.on_epoch_end(epoch_idx -1, batch_logs)
model.epoch = max(epoch_idx - 1, epochs)

print('Final : self.epoch {}   epochs {}'.format(model.epoch, epochs))
# end if (else)

## Fine Tuning
Fine tune all layers

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=211,
            layers="all")

## Save 

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
model.keras_model.save_weights(model_path)